# Numerical demographic as intermediate fusion

In [1]:
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

from transformers import (
    DataCollatorWithPadding,
    AutoModel,
    AutoTokenizer
)
from datasets import Dataset

from sklearn.preprocessing import MinMaxScaler

os.chdir("/g/data/jr19/rh2942/text-empathy/")
from evaluation import pearsonr
from utils.utils import plot, get_device, set_all_seeds

In [2]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false' # due to huggingface warning
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [3]:
class RobertaRegressor(nn.Module):
    def __init__(self, checkpoint):
        super(RobertaRegressor, self).__init__()
        self.transformer = AutoModel.from_pretrained(checkpoint)
        self.pre_classifier = nn.Linear(768, 768)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(768, 512)
        self.pre_final = nn.Linear(517, 256)
        self.final = nn.Linear(256, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        # token_type_ids=None,
        gender=None,
        education=None,
        race=None,
        age=None,
        income=None
    ):

        output = self.transformer(
            input_ids= input_ids,
            attention_mask=attention_mask,
            # token_type_ids=token_type_ids
        )
        output = output[0]
        output = output[:, 0]
        output = self.pre_classifier(output)
        output = nn.ReLU()(output)
        output = self.dropout(output)
        output = self.classifier(output)
        output = nn.ReLU()(output)
        output = torch.cat([output, gender, education, race, age, income], 1)
        output = self.pre_final(output)
        output = nn.ReLU()(output)
        output = self.final(output)
        return output

In [4]:
class DataModule():
    def __init__(self, task, checkpoint, batch_size, max_len, feature_to_tokenise):
        super(DataModule, self).__init__()
        self.task = task
        self.checkpoint = checkpoint
        self.batch_size = batch_size
        self.max_len = max_len
        self.tokeniser = AutoTokenizer.from_pretrained(
            self.checkpoint,
            use_fast=True
        )
        self.data_collator = DataCollatorWithPadding(tokenizer=self.tokeniser)
        self.feature_to_tokenise = feature_to_tokenise # to tokenise function
    
    def _process_raw(self, path, send_label):
        data = pd.read_csv(path, sep='\t')
    
        if send_label:
            text = data[self.feature_to_tokenise + self.task]
        else:
            text = data[self.feature_to_tokenise]

        data_demog = data[demog]
        scaler = MinMaxScaler()
        data_demog = pd.DataFrame(
            scaler.fit_transform(data_demog),
            columns=demog
        )
        data = pd.concat([text, data_demog], axis=1) 
        return data

    def _tokeniser_fn(self, sentence):
        if len(self.feature_to_tokenise) == 1: # only one feature
            return self.tokeniser(sentence[self.feature_to_tokenise[0]], truncation=True)
        # otherwise tokenise a pair of sentence
        return self.tokeniser(sentence[self.feature_to_tokenise[0]], sentence[self.feature_to_tokenise[1]], truncation=True)

    def _process_input(self, file, send_label):
        data = self._process_raw(path=file, send_label=send_label)
        data = data.reset_index(drop=True)
        data = Dataset.from_pandas(data, preserve_index=False) # convert to huggingface dataset
        data = data.map(self._tokeniser_fn, batched=True, remove_columns=self.feature_to_tokenise) # tokenise
        data = data.with_format('torch')
        return data

    def dataloader(self, file, send_label, shuffle):
        data = self._process_input(file=file, send_label=send_label)
        return DataLoader(
            data,
            batch_size=self.batch_size,
            shuffle=shuffle,
            collate_fn=self.data_collator,
            num_workers=24
        )

In [5]:
class Trainer:
    def __init__(self, task, model, lr):
        self.device = get_device(0)
        self.task = task
        self.model = model.to(self.device)
        self.lr = lr
        self.loss_fn = nn.MSELoss()
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr = self.lr)
        self.best_pearson_r = -1.0 # initiliasation

    def _training_step(self, epoch, train_loader):
        tr_loss = 0.0
        self.model.train()
    
        for i, data in enumerate(train_loader, 0):
            input_ids = data['input_ids'].to(self.device, dtype=torch.long)
            attention_mask = data['attention_mask'].to(self.device, dtype=torch.long)
            # token_type_ids = data['token_type_ids'].to(self.device, dtype=torch.long)
            targets = data[self.task].to(self.device, dtype=torch.float).view(-1, 1)
            gender = data['gender'].to(self.device, dtype=torch.float).view(-1, 1)
            education = data['education'].to(self.device, dtype=torch.float).view(-1, 1)
            race = data['race'].to(self.device, dtype=torch.float).view(-1, 1)
            age = data['age'].to(self.device, dtype=torch.float).view(-1, 1)
            income = data['income'].to(self.device, dtype=torch.float).view(-1, 1)
    
            outputs = self.model(input_ids, attention_mask, gender, education, race, age, income)
            loss = self.loss_fn(outputs, targets)
            tr_loss += loss.item()
    
            if i % 50 == 0:
                loss_step = tr_loss / (i+1)
                print(f'Training loss per 50 steps: {loss_step}')
    
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
    
        epoch_loss = tr_loss / (i+1)
        print(f'Epoch: {epoch+1}')
        print(f'The total loss: {epoch_loss}')

    def fit(self, n_epochs, train_loader, dev_loader, output_label, save_model=False):
        set_all_seeds(0)
        for epoch in range(n_epochs):
            self._training_step(epoch, train_loader)

            preds = self.evaluate(dataloader=dev_loader, load_model=False)
            if self.task == 'empathy':
                true = output_label.iloc[:, 0].tolist()
            if self.task == 'distress':
                true = output_label.iloc[:, 1].tolist()
            pearson_r = pearsonr(true, preds)
            print(f'Pearson r: {pearson_r}')
            
            if save_model and (pearson_r > self.best_pearson_r):
                self.best_pearson_r = pearson_r   
                torch.save(self.model.state_dict(), 'roberta-empathy.pth')
                print("Saved the model in epoch " + str(epoch+1))

    def evaluate(self, dataloader, load_model=False):
        if load_model:
            self.model.load_state_dict(torch.load('roberta-empathy.pth'))
    
        pred = torch.empty((len(dataloader.dataset), 1), device=self.device) # len(self.dev_loader.dataset) --> # of samples
        self.model.eval()
    
        with torch.no_grad():
            idx = 0
            for data in dataloader:
                input_ids = data['input_ids'].to(self.device, dtype=torch.long)
                attention_mask = data['attention_mask'].to(self.device, dtype=torch.long)
                # token_type_ids = data['token_type_ids'].to(self.device, dtype=torch.long)
                gender = data['gender'].to(self.device, dtype=torch.float).view(-1, 1)
                education = data['education'].to(self.device, dtype=torch.float).view(-1, 1)
                race = data['race'].to(self.device, dtype=torch.float).view(-1, 1)
                age = data['age'].to(self.device, dtype=torch.float).view(-1, 1)
                income = data['income'].to(self.device, dtype=torch.float).view(-1, 1)
        
                outputs = model(input_ids, attention_mask, gender, education, race, age, income)

                batch_size = outputs.shape[0]
                pred[idx:idx+batch_size, :] = outputs
                idx += batch_size
            
        return [float(k) for k in pred]   

In [9]:
MAX_LEN = 256
BATCH_SIZE = 8
lr = 1e-05
checkpoint = 'roberta-base'
task = ['empathy']

train_file = './data/COMBINED-PREPROCESSED-PARAPHRASED-WS22-WS23-train.tsv'

# WASSA 2022
# dev_file = './data/PREPROCESSED-WS22-dev.tsv'
# dev_label_file = './data/WASSA22/goldstandard_dev_2022.tsv'
# test_file = './data/PREPROCESSED-WS22-test.tsv'

# WASSA 2023
dev_file = './data/PREPROCESSED-WS23-dev.tsv'
dev_label_file = './data/WASSA23/goldstandard_dev.tsv'
test_file = './data/PREPROCESSED-WS23-test.tsv'

demog = ['gender', 'education', 'race', 'age', 'income']

In [10]:
data_module = DataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=BATCH_SIZE,
    max_len=MAX_LEN,
    feature_to_tokenise=['demographic_essay']
)

In [11]:
train_loader = data_module.dataloader(file=train_file, send_label=True, shuffle=True)
dev_loader = data_module.dataloader(file=dev_file, send_label=False, shuffle=False)

Map:   0%|          | 0/5272 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [12]:
model = RobertaRegressor(checkpoint=checkpoint)

trainer = Trainer(
    task='empathy',
    model=model,
    lr=lr
)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
dev_label = pd.read_csv(dev_label_file, sep='\t', header=None)

In [14]:
trainer.fit(n_epochs=50, train_loader=train_loader, dev_loader=dev_loader,
            output_label=dev_label, save_model=True)

Training loss per 50 steps: 25.91261100769043
Training loss per 50 steps: 15.974660518122654
Training loss per 50 steps: 12.24951203270714
Training loss per 50 steps: 9.451612080169829
Training loss per 50 steps: 7.968791118901761
Training loss per 50 steps: 7.122134176383455
Training loss per 50 steps: 6.526921267921346
Training loss per 50 steps: 6.131153497770641
Training loss per 50 steps: 5.832774866102937
Training loss per 50 steps: 5.614562205755526
Training loss per 50 steps: 5.405586934494163
Training loss per 50 steps: 5.23293901668919
Training loss per 50 steps: 5.103282369809619
Training loss per 50 steps: 4.977959223331943
Epoch: 1
The total loss: 4.964337428802786
Pearson r: -0.003
Saved the model in epoch 1
Training loss per 50 steps: 2.4669947624206543
Training loss per 50 steps: 3.495080272356669
Training loss per 50 steps: 3.276456018485645
Training loss per 50 steps: 3.334510504409967
Training loss per 50 steps: 3.3285062920987905
Training loss per 50 steps: 3.264049

In [15]:
test_loader = data_module.dataloader(file=test_file, send_label=False, shuffle=False)
pred = trainer.evaluate(dataloader=test_loader, load_model=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
pred_df = pd.DataFrame({'emp': pred, 'dis': pred})

In [17]:
pred_df

,emp,dis
0,4.179225,4.179225
1,5.455332,5.455332
2,6.000626,6.000626
3,4.468859,4.468859
4,5.359300,5.359300
...,...,...
95,4.995355,4.995355
96,5.935928,5.935928
97,5.576462,5.576462
98,5.205278,5.205278


In [18]:
pred_df.to_csv('./tmp/predictions_EMP.tsv', sep='\t', index=None, header=None)

In [ ]:
true = dev_label.iloc[:,0].tolist()
pearsonr(true, pred)